In [1]:
import tensorflow as tf
import numpy as np
import pickle

In [10]:
train_data = pickle.load(open('../word2vec/anouymous_train_data_200.npz', 'rb'))
train_inputs = train_data['inputs']
train_targets = train_data['targets']

In [11]:
print(train_inputs['13_0'].shape)
print(train_targets['13_0'].shape)

(200, 13, 200)
(200, 134)


In [13]:
padded_inputs = dict()
padded_targets = dict()
max_batch = 200
max_step = 40
feature = 200
for key in train_inputs.keys():
    temp = train_inputs[key]
    temp2 = train_targets[key]
    if temp.shape[0] != max_batch and temp.shape[0] != 0:
#         print(temp.shape)
#         print(50-len(temp))
#         print(np.zeros((50 - temp.shape[0], temp.shape[1], 200)).shape)
        temp = np.concatenate((temp, np.zeros((max_batch - temp.shape[0], temp.shape[1], feature))), 0)
        temp2 = np.concatenate((temp2, np.zeros((max_batch - temp2.shape[0], 134))), 0)
#         print(temp.shape)
        temp = np.concatenate((temp, np.zeros((max_batch, max_step - temp.shape[1], feature))), 1)
        padded_inputs[key] = temp
        padded_targets[key] = temp2
    elif temp.shape[0] == max_batch and temp.shape[0] != 0:
        temp = np.concatenate((temp, np.zeros((max_batch, max_step - temp.shape[1], feature))), 1)
        padded_inputs[key] = temp
        padded_targets[key] = train_targets[key]

In [14]:
print(padded_inputs['13_1'].shape)
print(padded_targets['13_1'].shape)

(200, 40, 200)
(200, 134)


In [15]:
def init(shape):
    return tf.Variable(tf.truncated_normal(shape), tf.float32)

def lstm_layer(inputs, batch_size, step_size):
    shape = inputs.shape.as_list()
    batch_size = shape[0]
    step_size = shape[1]
    n_units = shape[2]
#     print(batch_size)
#     print(step_size)

    wf = init([n_units, n_units])
    bf = init([n_units])

    wi = init([n_units, n_units])
    bi = init([n_units])

    wc = init([n_units, n_units])
    bc = init([n_units])

    wo = init([n_units, n_units])
    bo = init([n_units])

    ht = init([batch_size, n_units])
    ct = init([1, n_units])

    ct_tiled = tf.tile(ct, [batch_size, 1])
        
    for i in range(step_size):
        xt = inputs[:, i, :]
        ft = tf.sigmoid(tf.matmul(ht, wf) + tf.matmul(xt, wf) + bf)
        it = tf.sigmoid(tf.matmul(ht, wi) + tf.matmul(xt, wi) + bi)
        c = tf.tanh(tf.matmul(ht, wc) + tf.matmul(xt, wc) + bc)
        ot = tf.sigmoid(tf.matmul(ht, wo) + tf.matmul(xt, wo) + bo)
        ct = tf.reshape(tf.reduce_mean(ft * ct_tiled + it * c, 0), [1, n_units])
        ht = ot * tf.tile(tf.tanh(ct), [batch_size, 1])
    
    return ht, ct

In [ ]:
graph = tf.Graph()
with graph.as_default():
    
    batch_size = 200
    step_size = 40
    feature = 200
    
    inputs_placeholder = tf.placeholder(tf.float32, [batch_size, step_size, feature])
    targets_placeholder = tf.placeholder(tf.float32, [batch_size, 134])
#     batch_size = tf.placeholder(tf.int32)
#     step_size = tf.placeholder(tf.int32)
    
#     outputs, states = tf.nn.dynamic_rnn(
#             cell = tf.contrib.rnn.BasicLSTMCell(num_units=200),
#             inputs = inputs_placeholder,
#             dtype = tf.float32)
        
#     print(outputs)
#     print(cells)
    outputs, states = lstm_layer(inputs_placeholder, batch_size, step_size)
    
    w1 = init([200, 300])
    b1 = init([300])
    
    w2 = init([300, 300])
    b2 = init([300])
    
    w3 = init([300, 134])
    b3 = init([134])
    
    o1 = tf.nn.relu(tf.add(tf.matmul(outputs, w1), b1))
    o2 = tf.nn.relu(tf.add(tf.matmul(o1, w2), b2))
    o3 = tf.nn.relu(tf.add(tf.matmul(o2, w3), b3))
    
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=o3, labels=targets_placeholder))
    sum_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=o3, labels=targets_placeholder))
    optimizer = tf.train.AdamOptimizer().minimize(loss)
    accuracy = tf.reduce_sum(tf.cast(tf.equal(tf.argmax(o3, 1), tf.argmax(targets_placeholder, 1)), tf.float32))
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    keys = padded_inputs.keys()
    
    for i in range(100):
        accs = 0.0
        errs = 0.0
        n = 0
        for key in keys:
            batch_inputs = padded_inputs[key]
            batch_targets = padded_targets[key]
            batch_size = len(train_inputs)
#             step_size = key.split('_')[0]
#             print(batch_inputs.shape)
#             print(batch_targets.shape)
            n += batch_size
            feed_dict = {inputs_placeholder:batch_inputs, targets_placeholder:batch_targets}#, batch_size:batch_size, step_size:step_size}
            _, acc, err = sess.run([optimizer, accuracy, sum_loss], feed_dict=feed_dict)
            accs += acc
            errs += err
            print('#Trainig Epoch: %d, ACC:%f ERR:%f'%(i, accs/n, errs/n))

#Trainig Epoch: 0, ACC:0.000000 ERR:9222.365994
#Trainig Epoch: 0, ACC:0.000000 ERR:8193.600144
#Trainig Epoch: 0, ACC:0.000000 ERR:8800.984150
#Trainig Epoch: 0, ACC:0.000000 ERR:8360.783501
#Trainig Epoch: 0, ACC:0.000000 ERR:8054.480692
#Trainig Epoch: 0, ACC:0.000000 ERR:7973.812560
#Trainig Epoch: 0, ACC:0.000000 ERR:8009.652944
#Trainig Epoch: 0, ACC:0.000000 ERR:8116.437410
#Trainig Epoch: 0, ACC:0.000000 ERR:8341.249520
#Trainig Epoch: 0, ACC:0.000000 ERR:8264.005259
#Trainig Epoch: 0, ACC:0.000000 ERR:8117.961161
#Trainig Epoch: 0, ACC:0.000000 ERR:8180.732889
#Trainig Epoch: 0, ACC:0.000000 ERR:8040.730104
#Trainig Epoch: 0, ACC:0.000000 ERR:7985.383131
#Trainig Epoch: 0, ACC:0.000000 ERR:7877.057012
#Trainig Epoch: 0, ACC:0.000000 ERR:7806.230232
#Trainig Epoch: 0, ACC:0.000000 ERR:7692.717028
#Trainig Epoch: 0, ACC:0.000000 ERR:7633.090778
#Trainig Epoch: 0, ACC:0.000000 ERR:7645.012134
#Trainig Epoch: 0, ACC:0.004539 ERR:7575.494524
#Trainig Epoch: 0, ACC:0.004323 ERR:7497

#Trainig Epoch: 0, ACC:0.002103 ERR:3066.274233
#Trainig Epoch: 0, ACC:0.002091 ERR:3051.851876
#Trainig Epoch: 0, ACC:0.002079 ERR:3039.358569
#Trainig Epoch: 0, ACC:0.002067 ERR:3024.949273
#Trainig Epoch: 0, ACC:0.002055 ERR:3011.815262
#Trainig Epoch: 0, ACC:0.002043 ERR:2997.231514
#Trainig Epoch: 0, ACC:0.002032 ERR:2984.831220
#Trainig Epoch: 0, ACC:0.002021 ERR:2971.554436
#Trainig Epoch: 0, ACC:0.002009 ERR:2958.377638
#Trainig Epoch: 0, ACC:0.001998 ERR:2945.658109
#Trainig Epoch: 0, ACC:0.001987 ERR:2932.992353
#Trainig Epoch: 0, ACC:0.001976 ERR:2919.060299
#Trainig Epoch: 0, ACC:0.001966 ERR:2908.866072
#Trainig Epoch: 0, ACC:0.001955 ERR:2897.481724
#Trainig Epoch: 0, ACC:0.001944 ERR:2885.610246
#Trainig Epoch: 0, ACC:0.001934 ERR:2872.740454
#Trainig Epoch: 0, ACC:0.001924 ERR:2862.974352
#Trainig Epoch: 0, ACC:0.001914 ERR:2852.307323
#Trainig Epoch: 0, ACC:0.001904 ERR:2841.750761
#Trainig Epoch: 0, ACC:0.001894 ERR:2829.482708
#Trainig Epoch: 0, ACC:0.001884 ERR:2818

#Trainig Epoch: 0, ACC:0.004214 ERR:1704.777242
#Trainig Epoch: 0, ACC:0.004201 ERR:1700.191877
#Trainig Epoch: 0, ACC:0.004189 ERR:1695.491668
#Trainig Epoch: 0, ACC:0.004177 ERR:1691.330461
#Trainig Epoch: 0, ACC:0.004165 ERR:1686.724600
#Trainig Epoch: 0, ACC:0.004153 ERR:1682.139797
#Trainig Epoch: 0, ACC:0.004385 ERR:1677.781210
#Trainig Epoch: 0, ACC:0.004372 ERR:1673.272492
#Trainig Epoch: 0, ACC:0.004360 ERR:1668.727984
#Trainig Epoch: 0, ACC:0.004347 ERR:1664.713238
#Trainig Epoch: 0, ACC:0.004335 ERR:1660.148502
#Trainig Epoch: 0, ACC:0.004323 ERR:1655.726268
#Trainig Epoch: 0, ACC:0.004311 ERR:1651.503663
#Trainig Epoch: 0, ACC:0.004298 ERR:1647.723110
#Trainig Epoch: 0, ACC:0.004286 ERR:1643.320279
#Trainig Epoch: 0, ACC:0.004274 ERR:1638.996855
#Trainig Epoch: 0, ACC:0.004263 ERR:1634.692766
#Trainig Epoch: 0, ACC:0.004251 ERR:1630.664830
#Trainig Epoch: 0, ACC:0.004239 ERR:1626.486788
#Trainig Epoch: 0, ACC:0.004227 ERR:1622.079413
#Trainig Epoch: 0, ACC:0.004216 ERR:1617

#Trainig Epoch: 0, ACC:0.004791 ERR:1167.501568
#Trainig Epoch: 0, ACC:0.004782 ERR:1165.286550
#Trainig Epoch: 0, ACC:0.004772 ERR:1163.080117
#Trainig Epoch: 0, ACC:0.004763 ERR:1160.882220
#Trainig Epoch: 0, ACC:0.004754 ERR:1158.746638
#Trainig Epoch: 0, ACC:0.004745 ERR:1156.605305
#Trainig Epoch: 0, ACC:0.004736 ERR:1154.432540
#Trainig Epoch: 0, ACC:0.004727 ERR:1152.268116
#Trainig Epoch: 0, ACC:0.004718 ERR:1150.111984
#Trainig Epoch: 0, ACC:0.004708 ERR:1148.029008
#Trainig Epoch: 0, ACC:0.004699 ERR:1145.926526
#Trainig Epoch: 0, ACC:0.004691 ERR:1143.814646
#Trainig Epoch: 0, ACC:0.004682 ERR:1141.714972
#Trainig Epoch: 0, ACC:0.004673 ERR:1139.599322
#Trainig Epoch: 0, ACC:0.004664 ERR:1137.491685
#Trainig Epoch: 0, ACC:0.004655 ERR:1135.512592
#Trainig Epoch: 0, ACC:0.004646 ERR:1133.476674
#Trainig Epoch: 0, ACC:0.004638 ERR:1131.506861
#Trainig Epoch: 0, ACC:0.004629 ERR:1129.482957
#Trainig Epoch: 0, ACC:0.004620 ERR:1127.527331
#Trainig Epoch: 0, ACC:0.004611 ERR:1125

#Trainig Epoch: 0, ACC:0.005522 ERR:886.692772
#Trainig Epoch: 0, ACC:0.005514 ERR:885.462570
#Trainig Epoch: 0, ACC:0.005912 ERR:884.216277
#Trainig Epoch: 0, ACC:0.005908 ERR:883.016331
#Trainig Epoch: 0, ACC:0.006312 ERR:881.777187
#Trainig Epoch: 0, ACC:0.006715 ERR:880.541623
#Trainig Epoch: 0, ACC:0.006706 ERR:879.311504
#Trainig Epoch: 0, ACC:0.007109 ERR:878.083054
#Trainig Epoch: 1, ACC:0.000000 ERR:26.767336
#Trainig Epoch: 1, ACC:0.000000 ERR:26.767336
#Trainig Epoch: 1, ACC:0.000000 ERR:58.710093
#Trainig Epoch: 1, ACC:0.000000 ERR:55.539189
#Trainig Epoch: 1, ACC:0.000000 ERR:49.784819
#Trainig Epoch: 1, ACC:0.000000 ERR:47.794233
#Trainig Epoch: 1, ACC:0.000000 ERR:44.790390
#Trainig Epoch: 1, ACC:0.000000 ERR:42.721560
#Trainig Epoch: 1, ACC:0.000000 ERR:44.530503
#Trainig Epoch: 1, ACC:0.000000 ERR:42.754186
#Trainig Epoch: 1, ACC:0.000000 ERR:41.300836
#Trainig Epoch: 1, ACC:0.000000 ERR:40.089711
#Trainig Epoch: 1, ACC:0.000000 ERR:39.064913
#Trainig Epoch: 1, ACC:0.0

#Trainig Epoch: 1, ACC:0.003792 ERR:40.355264
#Trainig Epoch: 1, ACC:0.003770 ERR:40.304864
#Trainig Epoch: 1, ACC:0.003748 ERR:40.226613
#Trainig Epoch: 1, ACC:0.003727 ERR:40.149261
#Trainig Epoch: 1, ACC:0.003705 ERR:40.072793
#Trainig Epoch: 1, ACC:0.003684 ERR:39.997193
#Trainig Epoch: 1, ACC:0.003663 ERR:39.922448
#Trainig Epoch: 1, ACC:0.003643 ERR:40.077625
#Trainig Epoch: 1, ACC:0.003622 ERR:40.003266
#Trainig Epoch: 1, ACC:0.003602 ERR:39.929733
#Trainig Epoch: 1, ACC:0.003582 ERR:39.951868
#Trainig Epoch: 1, ACC:0.003563 ERR:39.879425
#Trainig Epoch: 1, ACC:0.003543 ERR:39.807774
#Trainig Epoch: 1, ACC:0.003524 ERR:39.892330
#Trainig Epoch: 1, ACC:0.003505 ERR:39.821384
#Trainig Epoch: 1, ACC:0.003486 ERR:40.033167
#Trainig Epoch: 1, ACC:0.003467 ERR:39.962226
#Trainig Epoch: 1, ACC:0.003449 ERR:39.892041
#Trainig Epoch: 1, ACC:0.003431 ERR:39.882421
#Trainig Epoch: 1, ACC:0.003413 ERR:39.904638
#Trainig Epoch: 1, ACC:0.003395 ERR:39.835857
#Trainig Epoch: 1, ACC:0.003377 ER

#Trainig Epoch: 1, ACC:0.005488 ERR:38.562219
#Trainig Epoch: 1, ACC:0.005472 ERR:38.528616
#Trainig Epoch: 1, ACC:0.005457 ERR:38.534221
#Trainig Epoch: 1, ACC:0.005441 ERR:38.564255
#Trainig Epoch: 1, ACC:0.005426 ERR:38.530931
#Trainig Epoch: 1, ACC:0.005411 ERR:38.497794
#Trainig Epoch: 1, ACC:0.005395 ERR:38.464843
#Trainig Epoch: 1, ACC:0.005380 ERR:38.439026
#Trainig Epoch: 1, ACC:0.005365 ERR:38.406424
#Trainig Epoch: 1, ACC:0.005350 ERR:38.895579
#Trainig Epoch: 1, ACC:0.005335 ERR:38.964165
#Trainig Epoch: 1, ACC:0.005321 ERR:38.930379
#Trainig Epoch: 1, ACC:0.005939 ERR:38.896780
#Trainig Epoch: 1, ACC:0.006085 ERR:38.863365
#Trainig Epoch: 1, ACC:0.006068 ERR:38.868842
#Trainig Epoch: 1, ACC:0.006052 ERR:38.835687
#Trainig Epoch: 1, ACC:0.006035 ERR:38.802713
#Trainig Epoch: 1, ACC:0.006019 ERR:38.769919
#Trainig Epoch: 1, ACC:0.006003 ERR:38.737304
#Trainig Epoch: 1, ACC:0.005986 ERR:38.704865
#Trainig Epoch: 1, ACC:0.005970 ERR:38.672601
#Trainig Epoch: 1, ACC:0.005954 ER

In [ ]:
a = tf.constant(np.arange(10).reshape(2,5), dtype=tf.float32)
b = tf.sigmoid(a)
s = tf.Session()
print(s.run(b))

In [61]:
print(np.arange(10))

[0 1 2 3 4 5 6 7 8 9]
